In [56]:
import tensorflow as tf

In [57]:
vggnet = tf.keras.applications.VGG16(weights='imagenet')

In [58]:
# 이미지 파일을 넘파이 배열로 변환
from PIL import Image
import numpy as np
dog_png = Image.open('/content/cat.png')
dog_array = np.array(dog_png)
dog_array.shape


(224, 224, 3)

In [59]:
vgg_prep_dog = tf.keras.applications.vgg16.preprocess_input(dog_array)
vgg_prep_dog.shape

(224, 224, 3)

In [60]:
# 분류
prediction = vggnet.predict(vgg_prep_dog[np.newaxis,:])  # 배치차원 추가

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [61]:
np.argmax(prediction[0]),  prediction[0][208]

(np.int64(281), np.float32(3.6522097e-06))

In [62]:
from tensorflow.keras.applications.vgg16 import decode_predictions
decode_predictions(prediction)

[[('n02123045', 'tabby', np.float32(0.43275443)),
  ('n02124075', 'Egyptian_cat', np.float32(0.31127918)),
  ('n02123159', 'tiger_cat', np.float32(0.21606451)),
  ('n02971356', 'carton', np.float32(0.0035795602)),
  ('n03223299', 'doormat', np.float32(0.0031308173))]]

In [63]:
# ResNet
# 층을 깊게 쌓으면 -> 더 복잡한 패턴을 학습
# 네트웍이 깊어지면
  # 기울기 소실(소실 / 폭발)
# Residual Learning  잔차 연결
# x-> H(x)
# F(x) = H(x) - x  Residual 잔차를 학습
# 최종 출력 H(x) = F(x) + x  스킵커넥

In [64]:
# ResNet 모델
import keras
from keras import layers
inputs = layers.Input(shape=(224,224,3))
x = layers.ZeroPadding2D(padding=3)(inputs)
x = layers.Conv2D(64,kernel_size=7,strides=2)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.ZeroPadding2D(padding=1)(x)
x = layers.MaxPooling2D(pool_size=3,strides=2)(x)

In [65]:
def residual_block(x, filters, first_stride=1, conv_skip=False):
    skip_conn = x
    x = layers.Conv2D(filters=filters, kernel_size=1,
                      strides=first_stride)(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=filters, kernel_size=3,
                      padding='same')(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=filters*4, kernel_size=1)(x)
    x = layers.BatchNormalization(epsilon=1e-5)(x)
    # conv_skip이 True 이면 1x1 합성곱ㅇ르 사용해 채널 크기를 fillers*4로 늘려준다
    if conv_skip == True:
        skip_conn = layers.Conv2D(filters=filters*4, kernel_size=1,
                                  strides=first_stride)(skip_conn)
        skip_conn = layers.BatchNormalization(epsilon=1e-5)(skip_conn)
    x = layers.Add()([skip_conn, x])
    x = layers.Activation('relu')(x)
    return x

In [66]:
# 잔차 스택
def build_stack(x):
  #  첫번째 잔차스택의 첫번째 잔차 블럭만 스트라이드 1
  x = residual_stack(x, 3, 64, first_stride=1)
  for blocks, filters in [(4, 128), (6, 256), (3, 512)]:
      x = residual_stack(x, blocks, filters, first_stride=2)
  return x

def residual_stack(x, blocks, filters, first_stride=2):
  # 합성곱 스킵커넥션
  x = residual_block(x, filters, first_stride=first_stride, conv_skip=True)
  for _ in range(1, blocks):
      x = residual_block(x, filters, first_stride=1, conv_skip=False)
  return x

In [67]:
# ResNet 모델
x = build_stack(x)
x = layers.GlobalAveragePooling2D()(x)
output = layers.Dense(1000, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])